In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

2025-05-16 17:21:03.965296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747416064.173698      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747416064.239880      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import tensorflow as tf

# Force TensorFlow to use GPU 0
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Allow memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("GPU is set and memory growth is enabled.")
    except RuntimeError as e:
        print(e)

GPU is set and memory growth is enabled.


In [3]:
# Optional: check GPU
print("Available GPU:", tf.config.list_physical_devices('GPU'))

Available GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# Set dataset paths
train_dir = '/kaggle/input/fer2013/train'
val_dir = '/kaggle/input/fer2013/test'

In [6]:
# Image generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,             # Keep face structure
    zoom_range=0.1,                # Slight zoom
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True           # Only flip is very safe
)


val_datagen = ImageDataGenerator(rescale=1./255)

train = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

val = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
# Define CNN model
model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(48,48,1)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(128, (3,3), activation='relu'),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1747416169.019614      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [8]:
print(type(model))

<class 'keras.src.models.sequential.Sequential'>


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight
import numpy as np


# Save best model
checkpoint = ModelCheckpoint("b2_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
import os
num_workers = min(4, os.cpu_count() or 1)
# Train model
model.fit(train, validation_data=val, epochs=30, callbacks=[checkpoint])


Epoch 1/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.2323 - loss: 1.8936
Epoch 1: val_accuracy improved from -inf to 0.24714, saving model to b2_model.h5
449/449 ━━━━━━━━━━━━━━━━━━━━ 53s 105ms/step - accuracy: 0.2323 - loss: 1.8935 - val_accuracy: 0.2471 - val_loss: 1.8112
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.2541 - loss: 1.8032
Epoch 2: val_accuracy improved from 0.24714 to 0.29521, saving model to b2_model.h5
449/449 ━━━━━━━━━━━━━━━━━━━━ 45s 100ms/step - accuracy: 0.2541 - loss: 1.8032 - val_accuracy: 0.2952 - val_loss: 1.7417
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.2758 - loss: 1.7592
Epoch 3: val_accuracy improved from 0.29521 to 0.32014, saving model to b2_model.h5
449/449 ━━━━━━━━━━━━━━━━━━━━ 45s 101ms/step - accuracy: 0.2759 - loss: 1.7592 - val_accuracy: 0.3201 - val_loss: 1.6744
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.3083 - loss: 1.7062
Epoch 4: val_accuracy improved from 0.32014